In [ ]:
import pandas as pd
from pytrends.request import TrendReq
import time
import json

# Load the JSON file
with open('uptownfunk.json', 'r') as file:
    uptownfunk_data = json.load(file)

# Extract unique words from the JSON content
keywords = list(set(word.lower() for entry in uptownfunk_data if 'text' in entry for word in entry['text'].split()))
regions = [
    "IN" 
]
timeframe = "now 7-d"  

In [26]:
# Extract unique words from the JSON content, convert to lowercase, and create a set
print(set(keywords))

{'break', 'down', 'gold', 'hood', 'doh', 'out', 'man', 'jar', 'liquor', 'what', 'leave', 'gotta', 'myself,', 'sign', 'chucks', 'than', 'am', 'hollywood,', "it's", 'good', 'dragon', 'retire,', 'and', 'brag', 'them', "wildin'", 'so', 'girls', 'name,', 'hot', 'stop', 'say', 'lemme', 'watch,', 'get', 'wanna', 'hallelujah,', 'i', 'stretch', 'jackson,', 'said', 'julio,', "stylin',", "don't", 'harlem,', 'up,', 'up', 'show', 'something', 'just', 'know', 'then', 'dance,', 'own', 'you,', 'about', 'ride', 'called', 'flaunt', 'bitch,', 'white', 'put', 'believe', 'smoother', 'me', 'you', 'masterpieces', 'it', 'jump', "y'all", 'michelle', 'on,', 'straight', "'bout", 'who', 'to', 'fresh', 'saturday', 'my', 'tell', 'hit', 'come', 'doh,', 'the', 'ooh', 'damn', 'if', 'money', 'kiss', 'funk', 'hit,', 'too', 'it,', 'of', 'mississippi', 'minute', 'spot', 'give', 'city', 'â™ª', 'police', "i'm", 'saint', 'make', 'before', 'fireman', 'hey,', 'one,', 'that', 'take', "livin'", 'night,', 'got', 'damn,', 'with', 

In [27]:
def chunk_keywords(keywords, chunk_size):
    """
    Splits a list of keywords into chunks of a specified size.
    
    Args:
        keywords (list): List of keywords to split.
        chunk_size (int): Size of each chunk.
    
    Returns:
        list: List of keyword chunks.
    """
    for i in range(0, len(keywords), chunk_size):
        yield keywords[i:i + chunk_size]

In [28]:
def analyze_search_trends(keywords, regions, timeframe):
    """
    Analyzes search trends for given keywords and regions with delays to avoid rate limits.
    
    Args:
        keywords (list): List of keywords to analyze (up to 5 per request).
        regions (list): List of region codes (e.g., 'US', 'UK').
        timeframe (str): Timeframe for trends (e.g., 'today 5-y').
    
    Returns:
        dict: Trend data for each region.
    """
    pytrends = TrendReq(hl='en-US', tz=360)  # Initialize pytrends
    all_data = {}

    # Split keywords into chunks of 5
    keyword_chunks = chunk_keywords(keywords, chunk_size=5)

    for region in regions:
        for chunk in keyword_chunks:
            try:
                # Build payload and fetch data
                pytrends.build_payload(chunk, cat=0, timeframe=timeframe, geo=region)
                data = pytrends.interest_over_time()
                if not data.empty:
                    if region not in all_data:
                        all_data[region] = data
                    else:
                        all_data[region] = all_data[region].join(data, how='outer')
                
                # Pause for 5 seconds to respect rate limits
                time.sleep(5)
            
            except Exception as e:
                print(f"Error fetching data for {region} with keywords {chunk}: {e}")
                # Wait longer if an error occurs, then continue
                time.sleep(10)
    
    return all_data

In [29]:
# Execute analysis
trend_data = analyze_search_trends(keywords, regions, timeframe)

# Display results
for region, data in trend_data.items():
    print(f"Trends for {region}:")
    print(data.head())
    print("\n")

Error fetching data for IN with keywords ['break', 'gold', 'hood', 'down', 'doh']: The request failed: Google returned a response with code 429
Error fetching data for IN with keywords ['out', 'man', 'jar', 'liquor', 'what']: The request failed: Google returned a response with code 429
Error fetching data for IN with keywords ['leave', 'gotta', 'myself,', 'sign', 'chucks']: The request failed: Google returned a response with code 429
Error fetching data for IN with keywords ['than', 'am', 'hollywood,', "it's", 'good']: The request failed: Google returned a response with code 429
Error fetching data for IN with keywords ['dragon', 'retire,', 'and', 'brag', 'them']: The request failed: Google returned a response with code 429
Error fetching data for IN with keywords ["wildin'", 'so', 'girls', 'name,', 'hot']: The request failed: Google returned a response with code 429


KeyboardInterrupt: 